# Access S2S api directly

This notebook walks through an example of sending an API call directly without using the space2stats python library

In [1]:
from typing import Dict

import numpy as np
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
from lonboard import Map, ScatterplotLayer
from geojson_pydantic import Feature, Polygon

In [2]:
BASE_URL = "https://space2stats.ds.io"
FIELDS_ENDPOINT = f"{BASE_URL}/fields"
SUMMARY_ENDPOINT = f"{BASE_URL}/summary"

In [3]:
response = requests.get(FIELDS_ENDPOINT)
if response.status_code != 200:
    raise Exception(f"Failed to get fields: {response.text}")

available_fields = response.json()
print("Available Fields:", available_fields)

Available Fields: ['sum_pop_2020', 'ogc_fid', 'sum_pop_f_0_2020', 'sum_pop_f_10_2020', 'sum_pop_f_15_2020', 'sum_pop_f_1_2020', 'sum_pop_f_20_2020', 'sum_pop_f_25_2020', 'sum_pop_f_30_2020', 'sum_pop_f_35_2020', 'sum_pop_f_40_2020', 'sum_pop_f_45_2020', 'sum_pop_f_50_2020', 'sum_pop_f_55_2020', 'sum_pop_f_5_2020', 'sum_pop_f_60_2020', 'sum_pop_f_65_2020', 'sum_pop_f_70_2020', 'sum_pop_f_75_2020', 'sum_pop_f_80_2020', 'sum_pop_m_0_2020', 'sum_pop_m_10_2020', 'sum_pop_m_15_2020', 'sum_pop_m_1_2020', 'sum_pop_m_20_2020', 'sum_pop_m_25_2020', 'sum_pop_m_30_2020', 'sum_pop_m_35_2020', 'sum_pop_m_40_2020', 'sum_pop_m_45_2020', 'sum_pop_m_50_2020', 'sum_pop_m_55_2020', 'sum_pop_m_5_2020', 'sum_pop_m_60_2020', 'sum_pop_m_65_2020', 'sum_pop_m_70_2020', 'sum_pop_m_75_2020', 'sum_pop_m_80_2020', 'sum_pop_m_2020', 'sum_pop_f_2020']


In [4]:
AOIModel = Feature[Polygon, Dict]

# ~kenya
aoi = {
    "type": "Feature",
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [33.78593974945852, 5.115816884114494],
                [33.78593974945852, -4.725410543134203],
                [41.94362577283266, -4.725410543134203],
                [41.94362577283266, 5.115816884114494],
                [33.78593974945852, 5.115816884114494]
            ]
        ]
    },
    "properties": {
        "name": "Updated AOI"
    }
}
          
  

feat = AOIModel(**aoi)

In [5]:
# Define the Request Payload
request_payload = {
    "aoi": aoi,
    "spatial_join_method": "centroid",
    "fields": ["sum_pop_2020"], 
    "geometry": "point"
}

# Get Summary Data
response = requests.post(SUMMARY_ENDPOINT, json=request_payload)
if response.status_code != 200:
    raise Exception(f"Failed to get summary: {response.text}")

summary_data = response.json()
df = pd.DataFrame(summary_data)

In [6]:
df['geometry'] = df['geometry'].apply(lambda geom: shape(geom))
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
gdf

,hex_id,geometry,sum_pop_2020
0,866a4a017ffffff,POINT (36.40245 -0.10117),676.912804
1,866a4a01fffffff,POINT (36.29170 -0.39057),347.182722
2,866a4a04fffffff,POINT (39.40696 -2.61300),285.943490
3,866a4a057ffffff,POINT (35.34318 -1.13463),279.463912
4,866a4a05fffffff,POINT (34.80333 2.72332),235.258474
...,...,...,...
25801,86969ed97ffffff,POINT (35.56103 -0.82817),196.448391
25802,86969ed9fffffff,POINT (41.42538 3.56386),296.740893
25803,86969eda7ffffff,POINT (37.98080 -3.00927),154.818453
25804,86969edafffffff,POINT (39.24303 4.54271),157.875350


In [7]:
# Define custom breaks and corresponding RGBA colors
breaks = [0, 1, 1000, 10000, 50000, 100000, 200000, gdf["sum_pop_2020"].max()]
colors = np.array([
    [211, 211, 211, 255],  # Light gray for 0
    [255, 255, 0, 255],    # Yellow for 1-1000
    [255, 165, 0, 255],    # Orange for 1000-10000
    [255, 0, 0, 255],      # Red for 10000-50000
    [128, 0, 128, 255],    # Purple for 50000-100000
    [0, 0, 255, 255],      # Blue for 100000-200000
    [0, 0, 139, 255],      # Dark blue for 200000+
])

# Function to assign colors based on custom bins
def assign_color(value, breaks, colors):
    for i in range(len(breaks) - 1):
        if breaks[i] <= value < breaks[i + 1]:
            return colors[i]
    return colors[-1]  # In case value exceeds all breaks

# Map sum_pop_2020 values to colors using the custom function
gdf['color'] = gdf["sum_pop_2020"].apply(lambda x: assign_color(x, breaks, colors))
colors = np.uint8(gdf['color'].tolist())

# Create the scatterplot layer with the assigned colors
layer = ScatterplotLayer.from_geopandas(gdf, get_radius=2000, get_fill_color=colors)

m = Map(layer)
m

Map(layers=[ScatterplotLayer(get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x1681dd180>
[
  [
    2…